#### This notebook aims to develop a ResNet50 model to classify sheep breeds. The model scores over 93% with a standard deviation of 1.5-2%. It is a code implementation used in the paper:
[Ensemble Algorithm using Transfer Learning for Sheep Breed Classification](http://https://ieeexplore.ieee.org/document/9465609)

#### *A lots of wool!! Certainly this one is not meant for meat!! This is the Merino sheep, wool of which is one of the most expensive in the world!!!*
![](https://upload.wikimedia.org/wikipedia/commons/a/a1/Merino_sheep.png)

#### Let's dive deep into modelling...

### *Necessary Imports*

In [1]:
import numpy as np
import pandas as pd
import os
import time
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping

from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from keras import backend as k
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

### *Plot a beautiful sheep face... Well, at least I think they are cute!* 

In [1]:
img_path = '../input/sheep-breed-classification/SheepFaceImages/Marino/00.jpgMA8.jpg'
img = image.load_img(img_path, target_size=(224, 224))
plt.imshow(img)

# Convert the images to numpy array
x = image.img_to_array(img)
print (x.shape)
x = np.expand_dims(x, axis=0)
print (x.shape)
x = preprocess_input(x)
print('Input image shape:', x.shape)

### Load the images spread across different directories

In [1]:
PATH = '../input/sheep-breed-classification'
# Define data path
data_path = PATH + '/SheepFaceImages'
data_dir_list = os.listdir(data_path) # List containing names of sheep breeds

In [1]:
img_data_list = []
img_list_lengths = []
names = []

# labels = np.ones((num_of_samples,),dtype='int64')

for dataset in data_dir_list:
    # Load images in dataset
    img_list = os.listdir(data_path+'/'+ dataset)
    img_list_lengths.append(len(img_list))
    
    # Save the name of the dataset
    names.append(dataset) 

    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        img_path = data_path + '/'+ dataset + '/'+ img 
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        #print('Input image shape:', x.shape)
        img_data_list.append(x)


In [1]:
print(names)

In [1]:
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

### *Assign the labels to the images, don't mess up!!! Merino is for wool and Suffolk is for meat! Mistakes can be costly, after all!!*

In [1]:
num_classes = 4
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

In [1]:
# Assign correct label to the list of images
start = 0
for label in range(num_classes):
    end = start + img_list_lengths[label]
    labels[start : end] = label
    start = end

In [1]:
Y = np_utils.to_categorical(labels, num_classes)

In [1]:
#Shuffle the dataset
x, y = shuffle(img_data, Y, random_state=777)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=777)

In [1]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

In [1]:
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

del X_train, X_test, y_train, y_test, Y

### Here goes everyone is concerned about 😎... ***Training and Validation***

In [1]:
folds = 10

# Lists for storing training metrics per fold 
train_acc_per_fold = []
loss_per_fold = []

# Lists for storing test metrics per fold
val_acc_per_fold = []
val_prec_per_fold = []
val_rec_per_fold = []
val_f1_per_fold = []

kfold = KFold(n_splits=folds, shuffle=True)

callback = EarlyStopping(monitor='accuracy', patience=10)

image_input = Input(shape=(224, 224, 3))

fold_no = 1
for train, test in kfold.split(inputs, targets):
    
    y_test = targets[test]
    test_labels = np.argmax(y_test, axis=1)
    
    # Save the labels in a file
    pd.DataFrame(y_test).to_csv('resnet50_y_test_fold' + str(fold_no) + '.csv', index=False)
    
    # Load the ResNet50 model with imagenet weights
    model = ResNet50(input_tensor = image_input, weights = 'imagenet', include_top = True)
    
    # Add some more layers to accomodate to our needs of classifying the sheep
    last_layer = model.get_layer('avg_pool').output  
    x = BatchNormalization()(last_layer)  # Accuracy: 94.58 (with this layer included)
    #x = Dropout(0.2)(last_layer)           # Accuracy: 93.63 (with this layer included)
    x = Flatten(name='flatten')(x)         # Accuracy: 94.24
    out = Dense(num_classes, activation='softmax', name='output_layer')(x)
    custom_resnet50_model = Model(inputs=image_input, outputs=out)
    #custom_resnet50_model.summary()
    
    for layer in model.layers:
        layer.trainable = False
        
    # Compile the model
    custom_resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    print(f'Training for fold {fold_no} ...')
    
    t=time.time()
    hist = custom_resnet50_model.fit(inputs[train], targets[train], batch_size=32, epochs=200, verbose=0, callbacks=[callback])
    print('Training time: %s seconds' % (time.time() - t))
    
    # Evaluate the trained model
    scores = custom_resnet50_model.evaluate(inputs[test], targets[test], batch_size=10, verbose=0)
    
    # Make predictions on the validation set and flatten
    preds = custom_resnet50_model.predict(inputs[test], batch_size=10, verbose=0)
    preds_flat = np.argmax(preds, axis=1)
    
    # Calculate validation metrics
    val_acc = accuracy_score(test_labels, preds_flat)
    val_prec = precision_score(test_labels, preds_flat, average='weighted')
    val_rec = recall_score(test_labels, preds_flat, average='weighted')
    val_f1 = f1_score(test_labels, preds_flat, average='weighted')
    
    val_acc_per_fold.append(val_acc*100)
    val_prec_per_fold.append(val_prec*100)
    val_rec_per_fold.append(val_rec*100)
    val_f1_per_fold.append(val_f1*100)
    
    # Save the scores and the validation predictions foldwise (if needed)
    pd.DataFrame(preds).to_csv('resnet50_preds_fold' + str(fold_no) + '.csv', index=False)
    pd.DataFrame(scores).to_csv('resnet50_history_fold' + str(fold_no) + '.csv', index=False)
    
    print(f'Training score for fold {fold_no}: {custom_resnet50_model.metrics_names[0]} of {scores[0]}; {custom_resnet50_model.metrics_names[1]} of {scores[1]*100}%')
    train_acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    fold_no += 1

### Here's what managers are concerned about 😅... ***Classification metrics***

In [1]:
# Average Scores for training and validation
print('------------------------------------------------------------------------')
print('Training score per fold')
for i in range(0, len(train_acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {train_acc_per_fold[i]}%')
print('------------------------------------------------------------------------')

print('Validation score per fold')
for i in range(0, len(val_acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Accuracy: {val_acc_per_fold[i]}% - Precision: {val_prec_per_fold[i]}% - Recall: {val_rec_per_fold[i]}% - F1 Score: {val_f1_per_fold[i]}%')
print('------------------------------------------------------------------------')

print('Average training scores for all folds:')
print(f'> Accuracy: {np.mean(train_acc_per_fold)}% (+- {np.std(train_acc_per_fold)}%)')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

print('Average validation scores for all folds:')
print(f'> Accuracy: {np.mean(val_acc_per_fold)}% (+- {np.std(val_acc_per_fold)}%)')
print(f'> Precision: {np.mean(val_prec_per_fold)}% (+- {np.std(val_prec_per_fold)}%)')
print(f'> Recall: {np.mean(val_rec_per_fold)}% (+- {np.std(val_rec_per_fold)}%)')
print(f'> F1 Score: {np.mean(val_f1_per_fold)}% (+- {np.std(val_f1_per_fold)}%)')
print('------------------------------------------------------------------------')

In [1]:
# resnet50_df = pd.DataFrame({'0': preds[:, 0], '1': preds[:, 1], '2': preds[:, 2], '3': preds[:, 3]})
# resnet50_df.to_csv('resnet50_predictions.csv', index=False)